In [13]:
import model as st_model
import dataset.stdatasets as st_data

# Models

In [14]:
# Users can load a default summarization model
model = st_model.summarizer()

# Or a specific model
pegasus = st_model.pegasus()

### Model selection

In [15]:
# Users can easily access documentation to assist with model selection
model.show_capability()

Pegasus is the default singe-document summarization model.
Introduced in 2019, a large neural abstractive summarization model trained on web crawl and news data.
 Strengths: 
 - High accuracy 
 - Performs well on almost all kinds of non-literary written text 
 Weaknesses: 
 - High memory usage 
 Initialization arguments: 
 - `device = 'cpu'` specifies the device the model is stored on and uses for computation. Use `device='gpu'` to run on an Nvidia GPU.


### Inference

In [16]:
documents = [
    """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. 
    The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected 
    by the shutoffs which were expected to last through at least midday tomorrow."""
]

model.summarize(documents)

["California's largest electricity provider has turned off power to hundreds of thousands of customers."]

# Datasets

In [18]:
cnn_dataset = st_data.load_dataset('cnn_dailymail', '3.0.0')

Reusing dataset cnn_dailymail (/home/zhangir/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)


In [19]:
cnn_dataset['train']['highlights'][0]

'Syrian official: Obama climbed to the top of the tree, "doesn\'t know how to get down"\nObama sends a letter to the heads of the House and Senate .\nObama to seek congressional approval on military action against Syria .\nAim is to determine whether CW were used, not by whom, says U.N. spokesman .'